# Uniaxial anisotropy energy term

> Interactive online tutorial:
> [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ubermag/oommfc/master?filepath=docs%2Fipynb%2Findex.ipynb)

Uniaxial anisotropy energy density is computed as

$$w_\text{a} = -K (\mathbf{m}\cdot\mathbf{u})^{2}$$

where $\mathbf{m}$ is the normalised ($|\mathbf{m}|=1$) magnetisation, $K$ is the uniaxial anisotropy constant, and $\mathbf{u}$ is the anisotropy axis. Uniaxial anisotropy energy term tends to align all magnetic moments parallel or antiparallel to the anisotropy axis.

In `oommfc`, $\mathbf{m}$ is a part of the magnetisation field `system.m`. Therefore, only uniaxial anisotropy constant $K$ and axis $\mathbf{u}$ should be provided as input parameters to uniquely define the energy term. Both $K$ and $\mathbf{u}$ can be constant in space or spatially varying.

## Spatially constant $K$ and $\mathbf{u}$

Let us start by assembling a simple simple simulation where neither $K$ nor $\mathbf{u}$ vary in space. The sample is a "one-dimensional" chain of magnetic moments.

In [1]:
import oommfc as oc

p1 = (-10e-9, 0, 0)
p2 = (10e-9, 1e-9, 1e-9)
cell = (1e-9, 1e-9, 1e-9)
mesh = oc.Mesh(p1=p1, p2=p2, cell=cell)

The mesh is

In [2]:
mesh.k3d()

Output()

The system has a Hamiltonian, which consists of only uniaxial anisotropy energy term.

In [3]:
K = 1e5  # uniaxial anisotropy constant (J/m**3)
u = (0, 0, 1)  # uniaxial anisotropy axis
system = oc.System(name='uniaxialanisotropy-constant-K-u')
system.hamiltonian = oc.UniaxialAnisotropy(K1=K, u=u)

We are going to minimise the system's energy using `oommfc.MinDriver` later. Therefore, we do not have to define the system's dynamics equation. Finally, we need to define the system's magnetisation (`system.m`). We are going to make it random with $M_\text{s}=8\times10^{5} \,\text{Am}^{-1}$

In [4]:
import random
import discretisedfield as df

Ms = 8e5  # saturation magnetisation (A/m)

def m_fun(pos):
    return [2*random.random()-1 for i in range(3)]

system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

The magnetisation, we set is

In [5]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

Now, we can minimise the system's energy by using `oommfc.MinDriver`.

In [6]:
md = oc.MinDriver()
md.drive(system)

2019/08/20 13:05: Running OOMMF (uniaxialanisotropy-constant-K-u.mif) ... (1.0 s)


We expect that now all magnetic moments are aligned parallel or antiparallel to the anisotropy axis (in the $z$-direction).

In [7]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

Finally, we can delete the files created by `oommfc`.

In [8]:
system.delete()

## Spatially varying $\mathbf{K}$

There are two different ways how a parameter can be made spatially varying, by using:
1. Dictionary
2. `discretisedfield.Field`

### Dictionary

In order to define a parameter using a dictionary, regions must be defined in the mesh. Regions are defined as a dictionary, whose keys are the strings and values are `discretisedfield.Region` objects, which take two corner points of the region as input parameters. 

In [9]:
p1 = (-10e-9, 0, 0)
p2 = (10e-9, 1e-9, 1e-9)
cell = (1e-9, 1e-9, 1e-9)
regions = {'region1': df.Region(p1=(-10e-9, 0, 0), p2=(0, 1e-9, 1e-9)),
           'region2': df.Region(p1=(0, 0, 0), p2=(10e-9, 1e-9, 1e-9))}
mesh = oc.Mesh(p1=p1, p2=p2, cell=cell, regions=regions)

In [10]:
mesh.k3d_regions()

Output()

Let us say that there is no uniaxial anisotropy ($K=0$) in region 1, whereas in region 2 it is $K=10^{5} \,\text{Jm}^{-3}$. $\mathbf{u}$ is in the $z$-direction. `K` is now defined as a dictionary:

In [11]:
K = {'region1': 0, 'region2': 1e5}

The system object is

In [12]:
system = oc.System(name='uniaxialanisotropy-dict-K')
system.hamiltonian = oc.UniaxialAnisotropy(K1=K, u=u)
system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

Its magnetisation is

In [13]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

After we minimise the energy

In [14]:
md.drive(system)

2019/08/20 13:05: Running OOMMF (uniaxialanisotropy-dict-K.mif) ... (1.0 s)


The magnetisation is as we expected.

In [15]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

In [16]:
system.delete()

### `discretisedfield.Field`

Let us define the spatailly varying uniaxial anisotropy, so that

$\mathbf{u}(x, y, z) = \left\{
\begin{array}{ll}
(0, 0, 1) & x \le 0 \\
(1, 0, 0) & x > 0 \\
\end{array}
\right. $

The value of `u` for the spatially varying anisotropy is set using a Python function.

In [17]:
def u_fun(pos):
    x, y, z = pos
    if x <= 0:
        return (0, 0, 1)
    else:
        return (1, 0, 0)

The uniaxial anisotropy parameters are

In [18]:
K = 1e5
u = df.Field(mesh, dim=3, value=u_fun)

The system is

In [19]:
system = oc.System(name='uniaxialanisotropy-field-u')
system.hamiltonian = oc.UniaxialAnisotropy(K1=K, u=u)
system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

and its magnetisation is

In [20]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

After the energy minimisation, the magnetisation is:

In [21]:
md.drive(system)
system.m.k3d_vectors(color_field=system.m.z)

2019/08/20 13:05: Running OOMMF (uniaxialanisotropy-field-u.mif) ... (1.0 s)


Output()

In [22]:
system.delete()

## Other

More details on various functionality can be found in the [API Reference](https://oommfc.readthedocs.io/en/latest/).